# Talabat Egypt Restaurant (Web Scraping)

### Import Libraries 

In [1]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2022-07-18 21:28:58--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 74.125.31.93, 74.125.31.91, 74.125.31.136, ...
Connecting to dl.google.com (dl.google.com)|74.125.31.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12415 (12K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  12.12K  --.-KB/s    in 0s      

2022-07-18 21:28:59 (51.6 MB/s) - ‘linux_signing_key.pub’ saved [12415/12415]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]       
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]    
Get:4 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1096 B]
Get:5 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [292 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                   

In [2]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2022-07-18 21:29:50--  http://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.253.123.128, 2607:f8b0:400c:c16::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.253.123.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7052799 (6.7M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   6.73M  --.-KB/s    in 0.1s    

2022-07-18 21:29:51 (60.9 MB/s) - ‘/tmp/chromedriver.zip’ saved [7052799/7052799]

Archive:  /tmp/chromedriver.zip
  inflating: /usr/local/bin/chromedriver  


In [10]:
# install selenium
!sudo apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 52 not upgraded.
Need to get 35.7 MB/36.4 MB of archives.
After this operation, 154 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.55.5+20.04 [35.6 MB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 chromium-browser amd64 1:85.0.4183.83-0ubuntu0.20.04.2 [48.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu0.20.04.2 [2496 B]
Ge

In [11]:
# To check Google Chrome's version
!google-chrome --version

Google Chrome 103.0.5060.114 


In [12]:
# To check Chrome Driver's version
!chromedriver -v

ChromeDriver 103.0.5060.53 (a1711811edd74ff1cf2150f36ffa3b0dae40b17f-refs/branch-heads/5060@{#853})


In [13]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
from PIL import Image

### Initialize &Open New Driver session 

In [14]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('-disk-cache-size-300000000000')
driver = webdriver.Chrome(options=chrome_options)


### Fetche Restaurants URL

In [15]:
driver.get(r'https://www.talabat.com/egypt/restaurants')

ElementsList=[]

In [16]:
try:
    while driver.find_element_by_css_selector("a[tabindex='0'][rel='next']"):
        Elements=driver.find_elements_by_css_selector("div[class='vendorstyles__VendorContainer-sc-eu6e2y-0 bPwOAu mx-2 mb-4']") 
        time.sleep(3)
        for job in Elements :
            link=job.find_element_by_css_selector("a")  
            ElementsList.append(link.get_attribute('href'))
        driver.find_element_by_css_selector("a[tabindex='0'][rel='next']").click()
except  :
    print('Links fetched')

Links fetched


In [17]:
len(ElementsList)

4920

### Create DataFrame

In [18]:
# Create DataFrame to store data
Restaurant_Data= pd.DataFrame(columns=["Restaurant",'Logo',"serving","Ratings","rating","Reviews",'OrderPackaging','ValueForMoney'
                                   ,'DeliveryTime','QualityOfFood'])
Restaurant_Data

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood


### Scraper & Store Data in DataFrame

In [19]:
for Restaurants in ElementsList :
    driver.get(Restaurants)
    time.sleep(2)
    try:
        Restaurant=(driver.find_element_by_css_selector("h1[class='text-center f-30 f-m-20 primary-text mt-2']").text)
        serving=(driver.find_element_by_css_selector("p[class='mt-2 text-center f-14 f-400 cuisine-string white']").text)
        Ratings=(driver.find_element_by_css_selector("div[class='ratings-count ml-1']").text)
        Logo=(driver.find_element_by_css_selector("img[class='brand-logo']").get_attribute("src")) 
        rating=(driver.find_element_by_css_selector("div[class='rating-number text-center mr-1']").text)
        Reviews=(driver.find_element_by_css_selector("span[class='f-500 ml-1']").text)
        OrderPackaging=(driver.find_element_by_css_selector('#__next > div:nth-child(4) > div.globalstyles__GlobalContainer-sc-fulyk5-0.beagJX > div > div > div.brandreviewsstyles__BrandReviewContainer-sc-6j7vf7-0.gBaayS.pt-6.b-t > div > div.restaurant-ratingstyles__RestaurantRatingContainer-sc-1hqlfwo-0.jXhrCP.mb-6.restaurant-rating-component > div > div:nth-child(1) > div > div > div').text)
        ValueForMoney=(driver.find_element_by_css_selector('#__next > div:nth-child(4) > div.globalstyles__GlobalContainer-sc-fulyk5-0.beagJX > div > div > div.brandreviewsstyles__BrandReviewContainer-sc-6j7vf7-0.gBaayS.pt-6.b-t > div > div.restaurant-ratingstyles__RestaurantRatingContainer-sc-1hqlfwo-0.jXhrCP.mb-6.restaurant-rating-component > div > div:nth-child(2) > div > div > div').text)
        DeliveryTime=(driver.find_element_by_css_selector('#__next > div:nth-child(4) > div.globalstyles__GlobalContainer-sc-fulyk5-0.beagJX > div > div > div.brandreviewsstyles__BrandReviewContainer-sc-6j7vf7-0.gBaayS.pt-6.b-t > div > div.restaurant-ratingstyles__RestaurantRatingContainer-sc-1hqlfwo-0.jXhrCP.mb-6.restaurant-rating-component > div > div:nth-child(3) > div > div > div').text)
        QualityOfFood=(driver.find_element_by_css_selector('#__next > div:nth-child(4) > div.globalstyles__GlobalContainer-sc-fulyk5-0.beagJX > div > div > div.brandreviewsstyles__BrandReviewContainer-sc-6j7vf7-0.gBaayS.pt-6.b-t > div > div.restaurant-ratingstyles__RestaurantRatingContainer-sc-1hqlfwo-0.jXhrCP.mb-6.restaurant-rating-component > div > div:nth-child(4) > div > div > div').text)

        Restaurant_Data = Restaurant_Data.append({"Restaurant":Restaurant,"Logo":Logo, "serving":serving, "Ratings":Ratings, "rating":rating,
        "Reviews":Reviews,"OrderPackaging":OrderPackaging,"ValueForMoney":ValueForMoney,
                            "DeliveryTime":DeliveryTime,"QualityOfFood":QualityOfFood}, ignore_index=True)
    except :
        print('Not found')

Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found


In [20]:
Restaurant_Data

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood
0,Broasty Restaurant,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Crepes, Fried chicken",( 29 Ratings ),3.5,34 Reviews,Good,Good,Very good,Good
1,{{brandname}},https://images.deliveryhero.io/image/talabat/r...,{{cuisine}},( 484 Ratings ),3.0,177 Reviews,Good,Good,Good,Good
2,Lo2ma Hanaya,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Egyptian",( 16 Ratings ),4.0,22 Reviews,Very good,Very good,Very good,Very good
3,Freshnfit,https://images.deliveryhero.io/image/talabat/r...,"Healthy , Smoothies, Desserts",( 102 Ratings ),4.5,36 Reviews,Very good,Very good,Very good,Very good
4,Kebda Fathy,https://images.deliveryhero.io/image/talabat/r...,Sandwiches,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK
...,...,...,...,...,...,...,...,...,...,...
4479,Abnaa Alhalabi,https://images.deliveryhero.io/image/talabat/r...,Desserts,( 181 Ratings ),4.0,69 Reviews,Very good,Very good,Very good,Very good
4480,Bless,https://images.deliveryhero.io/image/talabat/r...,Cosmetics,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK
4481,MOLOK EL GAMBRY,https://images.deliveryhero.io/image/talabat/r...,"Seafood, Sandwiches",( 774 Ratings ),4.0,320 Reviews,Very good,Very good,Very good,Very good
4482,Talah,https://images.deliveryhero.io/image/talabat/r...,Nuts & Dried Fruit,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK


### save data 

In [21]:
import os
os.chdir('/kaggle/working')
Restaurant_Data.to_csv('/kaggle/working/Talabat_Restaurant_Data.csv')


In [22]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/linux_signing_key.pub
/kaggle/working/__notebook_source__.ipynb
/kaggle/working/Talabat_Restaurant_Data.csv


In [23]:
Restaurant_df= pd.read_csv(r'/kaggle/working/Talabat_Restaurant_Data.csv',)
Restaurant_df=Restaurant_df.drop('Unnamed: 0',axis=1)
Restaurant_df

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood
0,Broasty Restaurant,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Crepes, Fried chicken",( 29 Ratings ),3.5,34 Reviews,Good,Good,Very good,Good
1,{{brandname}},https://images.deliveryhero.io/image/talabat/r...,{{cuisine}},( 484 Ratings ),3.0,177 Reviews,Good,Good,Good,Good
2,Lo2ma Hanaya,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Egyptian",( 16 Ratings ),4.0,22 Reviews,Very good,Very good,Very good,Very good
3,Freshnfit,https://images.deliveryhero.io/image/talabat/r...,"Healthy , Smoothies, Desserts",( 102 Ratings ),4.5,36 Reviews,Very good,Very good,Very good,Very good
4,Kebda Fathy,https://images.deliveryhero.io/image/talabat/r...,Sandwiches,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK
...,...,...,...,...,...,...,...,...,...,...
4479,Abnaa Alhalabi,https://images.deliveryhero.io/image/talabat/r...,Desserts,( 181 Ratings ),4.0,69 Reviews,Very good,Very good,Very good,Very good
4480,Bless,https://images.deliveryhero.io/image/talabat/r...,Cosmetics,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK
4481,MOLOK EL GAMBRY,https://images.deliveryhero.io/image/talabat/r...,"Seafood, Sandwiches",( 774 Ratings ),4.0,320 Reviews,Very good,Very good,Very good,Very good
4482,Talah,https://images.deliveryhero.io/image/talabat/r...,Nuts & Dried Fruit,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK


### Data Cleaning

In [24]:
display(Restaurant_Data.duplicated().sum())
Restaurant_Data.drop_duplicates(inplace=True)
display(Restaurant_Data.duplicated().sum())
display(Restaurant_Data.head())

58

0

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood
0,Broasty Restaurant,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Crepes, Fried chicken",( 29 Ratings ),3.5,34 Reviews,Good,Good,Very good,Good
1,{{brandname}},https://images.deliveryhero.io/image/talabat/r...,{{cuisine}},( 484 Ratings ),3.0,177 Reviews,Good,Good,Good,Good
2,Lo2ma Hanaya,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Egyptian",( 16 Ratings ),4.0,22 Reviews,Very good,Very good,Very good,Very good
3,Freshnfit,https://images.deliveryhero.io/image/talabat/r...,"Healthy , Smoothies, Desserts",( 102 Ratings ),4.5,36 Reviews,Very good,Very good,Very good,Very good
4,Kebda Fathy,https://images.deliveryhero.io/image/talabat/r...,Sandwiches,( 0 Ratings ),0.0,0 Reviews,OK,OK,OK,OK


In [25]:
#Split Ratings as number without "Ratings"
result = Restaurant_df['Ratings'].str.replace(' ','').str.split('(\d+)([A-Za-z]+)', expand=True)
Restaurant_df['Ratings']=result[1]
Restaurant_df.head()

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood
0,Broasty Restaurant,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Crepes, Fried chicken",29,3.5,34 Reviews,Good,Good,Very good,Good
1,{{brandname}},https://images.deliveryhero.io/image/talabat/r...,{{cuisine}},484,3.0,177 Reviews,Good,Good,Good,Good
2,Lo2ma Hanaya,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Egyptian",16,4.0,22 Reviews,Very good,Very good,Very good,Very good
3,Freshnfit,https://images.deliveryhero.io/image/talabat/r...,"Healthy , Smoothies, Desserts",102,4.5,36 Reviews,Very good,Very good,Very good,Very good
4,Kebda Fathy,https://images.deliveryhero.io/image/talabat/r...,Sandwiches,0,0.0,0 Reviews,OK,OK,OK,OK


In [26]:
#Split Reviews as number without "Reviews"
result = Restaurant_df['Reviews'].str.replace(' ','').str.split('(\d+)([A-Za-z]+)', expand=True)
Restaurant_df['Reviews']=result[1]
Restaurant_df

,Restaurant,Logo,serving,Ratings,rating,Reviews,OrderPackaging,ValueForMoney,DeliveryTime,QualityOfFood
0,Broasty Restaurant,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Crepes, Fried chicken",29,3.5,34,Good,Good,Very good,Good
1,{{brandname}},https://images.deliveryhero.io/image/talabat/r...,{{cuisine}},484,3.0,177,Good,Good,Good,Good
2,Lo2ma Hanaya,https://images.deliveryhero.io/image/talabat/r...,"Sandwiches, Egyptian",16,4.0,22,Very good,Very good,Very good,Very good
3,Freshnfit,https://images.deliveryhero.io/image/talabat/r...,"Healthy , Smoothies, Desserts",102,4.5,36,Very good,Very good,Very good,Very good
4,Kebda Fathy,https://images.deliveryhero.io/image/talabat/r...,Sandwiches,0,0.0,0,OK,OK,OK,OK
...,...,...,...,...,...,...,...,...,...,...
4479,Abnaa Alhalabi,https://images.deliveryhero.io/image/talabat/r...,Desserts,181,4.0,69,Very good,Very good,Very good,Very good
4480,Bless,https://images.deliveryhero.io/image/talabat/r...,Cosmetics,0,0.0,0,OK,OK,OK,OK
4481,MOLOK EL GAMBRY,https://images.deliveryhero.io/image/talabat/r...,"Seafood, Sandwiches",774,4.0,320,Very good,Very good,Very good,Very good
4482,Talah,https://images.deliveryhero.io/image/talabat/r...,Nuts & Dried Fruit,0,0.0,0,OK,OK,OK,OK
